### <center>2018 Winter CS101.08</center>

# <center>散列表和集合</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 今日议程

1. 映射和字典
2. 散列表
3. 集合

## 映射和字典

- 映射（Mapping）可以理解通过一个函数过程将两个（类）对象一一对应起来。
- 字典（Dictionary）通常指可以实现映射功能的一种数据结构，Python中用dict关键字或‘{}’来表示，在其他语言中也被称作Map。



### Python中的通用映射类型

- collections.abc中提供了Mapping和MutableMapping，描述了dict和其衍生类的实现接口
    - collections.defaultdict
    - collections.OrderedDict
    - collections.ChainMap
    - collections.Counter
    - collections.UserDict

![](img/8-1.png)

In [ ]:
from collections.abc import MutableMapping
a = dict(a=1, b=2, c=3)
b = {'a': 1, 'b': 2, 'c': 3}
c = dict(zip(['a', 'b', 'c'], [1, 2, 3]))
d = dict([('a', 1), ('b', 2), ('c', 3)])
e = {k:v for v, k in enumerate(list('abc'), 1)}
print(a == b == c == d == e)
isinstance(d, MutableMapping)

In [ ]:
issubclass(dict, MutableMapping)

In [ ]:
import this

In [ ]:
import re
import collections

WORD_RE = re.compile(r'\w+')

# raise KeyError when using index = {}
index = collections.defaultdict(list)
with open('src/ch08/zen_of_python.txt', encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)

            index[word].append(location)

for word in sorted(index, key=str.upper):
    print(word, index[word])

In [ ]:
words = re.findall(r'\w+', open('src/ch08/zen_of_python.txt').read().lower())
collections.Counter(words).most_common(10)

In [ ]:
class LRU(collections.OrderedDict):
    'Limit size, evicting the least recently looked-up key when full'

    def __init__(self, maxsize=4, *args, **kwds):
        self.maxsize = maxsize
        super().__init__(*args, **kwds)

    def __getitem__(self, key):
        value = super().__getitem__(key)
        self.move_to_end(key)
        return value

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        if len(self) > self.maxsize:
            oldest = next(iter(self))
            del self[oldest]

lru = LRU()
for v, k in enumerate(list('abcd')):
    lru[k] = v
print(lru)

In [ ]:
print(lru['a'])
print(lru)

In [ ]:
lru['e'] = 4
print(lru)

In [ ]:
da = {k:v for v, k in enumerate(list('abcd'))}
db = {k:v for v, k in enumerate(list('xyz'))}
print(da)
print(db)
cm = collections.ChainMap(da, db)
cm

In [ ]:
print(cm['a'], cm['x'])
db['x'] = 10
cm

In [ ]:
cm['a'] = 100
cm

In [ ]:
class WeakDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value + value)
        
wd = WeakDict(a=1)
wd

In [ ]:
wd['b'] = 2
wd

In [ ]:
wd.update({'c': 3})
wd

In [ ]:
class MyDict(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value + value)

md = MyDict(a=1)
md['b'] = 2
md.update({'c': 3})
md

### Python中dict的映射方法

![](img/8-2.png)

### Immutable Mapping

Python3.3的types包提供了一个MappingProxyType类型，可以对一个dict提供只读的封装

In [ ]:
from types import MappingProxyType
d = {'A': 1}
dp = MappingProxyType(d)
dp

In [ ]:
dp['A']

In [ ]:
dp['B'] = 2

In [ ]:
d['B'] = 2
dp

## 散列表

散列表（Hash Table）也称哈希表，可以看做是数组结构的扩展，支持按键值或关键字随机访问数据

- 查找操作的时间复杂度为O(1)
- 把关键字转换成数组索引的函数称为散列函数或哈希函数
- 

![](img/8-3.jpg)

### Python中的hash函数

> An object is hashable if it has a hash value which never changes during its lifetime (it needs a __hash__() method), and can be compared to other objects (it needs an __eq__() method). Hashable objects which compare equal must have the same hash value.

- 原子的immutable类型，如str，bytes，数字类型都是可以hash的
- forzenset是可以hansh的，因为它包含的元素必须是可以hash的
- tuple只有当包含的元素都是可hash的元素时，整个tuple才是可以hash的
- 自定义的类型默认是可以hash的，其值和id()相关

In [ ]:
tt = (1, 2, (30, 40))
hash(tt)

In [ ]:
tl = (1, 2, [30, 40])
hash(tl)

In [ ]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

### 散列冲突

对不同的关键字可能得到同一散列地址，即 $k_{1}\neq k_{2}$，而 $ hash(k_{1})=hash(k_{2})$，这种情况称为散列冲突或哈希冲突(hash collision)

![](img/8-4.png)



#### 解决冲突的方法：

- 开放寻址法(Open Addressing)

    - 线性探测(Linear probing)
    
    ![](img/8-5.png)
    
    - 二次探测(Quadratic probing)
    
    ![](img/8-6.png)

- 开放寻址法(Open Addressing)

    - 双重散列(Double hashing)
    
    ![](img/8-7.png)
    
- 链表法(Separate Chaining)
    
    ![](img/8-8.png)

### 如何设计散列表

- 散列函数不仅要简单，而且其生成的值尽可能随机且均匀的分布
    - [MurmurHash](https://github.com/aappleby/smhasher)
    - Google [CityHash](https://github.com/google/cityhash) [FarmHash](https://github.com/google/farmhash)
    - [CLHash](https://github.com/lemire/clhash)
- 适时地进行动态的扩容和缩容，减少散列冲突的发生和高效利用内存空间
    - 装载因子 = 填入表中的元素个数 / 散列表的长度
- 设计高效的扩容策略，比如分批迁移数据到新的扩容空间
- 选择合适的散列冲突解决方案：
    - 开放寻址需要维持装载因子不能过高，适合数据量较小或者需要序列化的对象
    - 链表法适合存储大对象和大数据量的情况，甚至可以将链表升级成更高级的数据结构


### Python中dict的散列表

#### dict所用散列表的一些特点

- dict的散列表在结构上是一个稀疏数组(Sparse Array)，数组的每个单元(Cell)称为桶(Bucket)或槽(Slot)
- 每个桶中存储的元素包括三个成员(hash, key, value)
- Python的中的策略是维持至少1/3的桶是空的，如果散列表的值太多就会动态的分配新的空间，并将原表拷贝过来
- 散列表中每个元素的分配位置都是通过内置的函数hash()来计算得到的
- CPython遇到散列冲突时采用开放寻址的方式来定位空的插入位置，但是探测的公式较复杂可以理解为随机探测(Random probing)
 

#### 使用dict的一些经验

- dick的key必须是可以哈希的对象
- dict的查询速度非常快，但相应的内存空间利用率低
- dict的key顺序和插入的次序相关，而且增加key可能会改变现有的顺序*
    - 所以当你遍历一个dict时，注意是否存在同时有并发的逻辑在修改dict
- 在Python3中，d.keys(), d.items(), d.values()分别会返回相应的view，它们类似与set而且是随着dict的变化而动态更新的

## 集合

集合(Set)是一种无序且不重复的元素合集

python中使用set和frozenset来表示数学上集合的概念：

- 最基本的功能是去重
- 因为底层也是用散列表所实现，所以要求每个元素都必须是可hash的
- set本身不能被hash，但是frozenset可以hash

In [ ]:
l = ['foo', 'bar', 'foo', 'baz']
set(l)

In [ ]:
s = {1}
type(s)

In [ ]:
hash(s)

In [ ]:
fs = frozenset(range(10))
hash(fs)

In [ ]:
s = {x for x in range(3)} #setcomps
s

In [ ]:
s.add(fs)
s

### Set的抽象类

![](img/8-9.png)

### Set的基本操作

#### 数学操作符

![](img/8-10.png)

![](img/8-11.png)

#### Set和Frozenset的方法

![](img/8-12.png)

# Any Questions?

## 课后作业 Assignment-06

1) 给出两个字符串s和t，判断这两个字符串是否采用相似的拼写模式。具体见下面的例子：

>Example 1:  
Input: s = "add", t = "tee"  
Output: true

>Example 2:  
Input: s = "man", t = "mom"  
Output: false

>Example 3:  
Input: s = "paper", t = "title"  
Output: true

注：两个字符串s和t是等长的。